# Heart Disease Prediction Project

+ The project involves prediction of heart disease (coronary heart disease) given health and socioeconomic data as inputs.

In [2]:
import pandas as pd
import numpy as np

# Casting The correct datatypes
data_types = {"male":np.bool_, "age":np.int0, "currentSmoker":np.bool_, "TenYearCHD": np.bool_ , "prevalentHyp":np.bool_}
main_df = pd.read_csv("dataset.csv", dtype= data_types)

main_df

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,True,39,4.0,False,0.0,0.0,0,False,0,195.0,106.0,70.0,26.97,80.0,77.0,False
1,False,46,2.0,False,0.0,0.0,0,False,0,250.0,121.0,81.0,28.73,95.0,76.0,False
2,True,48,1.0,True,20.0,0.0,0,False,0,245.0,127.5,80.0,25.34,75.0,70.0,False
3,False,61,3.0,True,30.0,0.0,0,True,0,225.0,150.0,95.0,28.58,65.0,103.0,True
4,False,46,3.0,True,23.0,0.0,0,False,0,285.0,130.0,84.0,23.10,85.0,85.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,True,50,1.0,True,1.0,0.0,0,True,0,313.0,179.0,92.0,25.97,66.0,86.0,True
4234,True,51,3.0,True,43.0,0.0,0,False,0,207.0,126.5,80.0,19.71,65.0,68.0,False
4235,False,48,2.0,True,20.0,NaN,0,False,0,248.0,131.0,72.0,22.00,84.0,86.0,False
4236,False,44,1.0,True,15.0,0.0,0,False,0,210.0,126.5,87.0,19.16,86.0,NaN,False


---
## EDA and Statistics
+ We will use the pandas profiler library for this

In [ ]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
from pandas_profiling import ProfileReport

profile = ProfileReport(main_df, title="Profiling Report", explorative=True)
profile.to_file("EDAreport.html")


  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

***

## Data Pre-processing

### Missing Data Imputation

In [3]:
from sklearn.impute import KNNImputer

# Separate out target attribute from the rest of the attributes
ten_year_chd = main_df.pop("TenYearCHD").astype("int")

# Imputation of various attributes that have missing values 
# Glucose Imputation - we will the top 3 most co-related features and use the kNN method for imputation 
gluc_temp_df = main_df[["glucose", "sysBP","age", "diabetes"]]
imputed_values = KNNImputer(n_neighbors=5, weights="uniform").fit_transform(gluc_temp_df)
main_df["glucose"] = imputed_values[:,0]


# for the rest of the missing values, since percent missing is insignificant, we just use kNN imputer
new_df = pd.DataFrame(KNNImputer(n_neighbors=5, weights="uniform").fit_transform(main_df))
new_df.columns = main_df.columns
main_df = new_df
main_df.index







RangeIndex(start=0, stop=4238, step=1)

In [ ]:
profile1 = ProfileReport(main_df, title="Post Processing", explorative=True)
profile1.to_file("PostProcessing.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

***
## Modelling (SVM and Logistic)
### 1. Logistic Regression Model

In [4]:
"""We will be using k-fold cross-validation instead of single hold-out(train-test split)"""
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
logisitic_model = LogisticRegression(solver='liblinear', C=3.0, random_state=0)
cross_val_score(logisitic_model, main_df, ten_year_chd, cv=15).mean()

0.8532324219565782

### 2. Support Vector Machine (SVM) Model

In [5]:
from sklearn import svm
svm_model = svm.SVC()
# Quick score of model performance
cross_val_score(svm_model, main_df, ten_year_chd, cv=15).mean()


0.8480415006390499

***
## Cross-validation and Evaluation

### 1. Cross-validation confusion matrix generation

In [6]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

# We have to generate confusion matrix manually for cross validation, then average the results
def get_confusion_matrix(model):
    conf_matrix_list_of_arrays = []
    skf = StratifiedKFold(n_splits=7, random_state=0, shuffle=True)
    for train_index, test_index in skf.split(main_df, ten_year_chd):
        X_train, X_test = main_df.iloc[train_index], main_df.iloc[test_index]
        y_train, y_test = ten_year_chd.iloc[train_index], ten_year_chd.iloc[test_index]
        model.fit(X_train, y_train)
        conf_matrix = confusion_matrix(y_test, model.predict(X_test))
        conf_matrix_list_of_arrays .append(conf_matrix)
    return np.round(np.mean(conf_matrix_list_of_arrays, axis=0), 4)
    
confusion_matrix_dict = {"logistic": get_confusion_matrix(logisitic_model), "svm": get_confusion_matrix(svm_model)}




### 2. Model Evaluation and Visualization


In [7]:
# We print the results of the model accuracy and view them graphically
!pip install plotly 

import plotly.graph_objects as go
from plotly.subplots import make_subplots

total_dict = {}
for key, val in confusion_matrix_dict.items():
    evaluation_dict = {}
    tn, fp, fn, tp = val.flatten()
    evaluation_dict["accuracy"] = (tp + tn) / (tp + tn + fn + fp)
    evaluation_dict["sensitivity"] = tp / (tp + fn)
    evaluation_dict["specificity"] = tn / (tn + fp)
    evaluation_dict["precision"] = tp / (tp + fp)
    print(f"{key}: {evaluation_dict}")
    total_dict[key] = {"values": (tn, fp, fn, tp), "metrics":evaluation_dict}


def generate_viz():
    fig = make_subplots(rows=2, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}],[{"colspan":2}, None]], subplot_titles=['Logisitic Model', 'SVM Model'])
    fig.update_layout(
    autosize=False,
    width=2000,
    height=1500,
)
    labels = ["True Negative", "False Positive", "False Negative", "True Positive"]
    trace1 = go.Bar(
    x=list(total_dict["logistic"]["metrics"].keys()),
    y=list(total_dict["logistic"]["metrics"].values()),
    name='Logistic'
)
    trace2 = go.Bar(
    x=list(total_dict["svm"]["metrics"].keys()),
    y=list(total_dict["svm"]["metrics"].values()),
    name='SVM'
)

    fig.add_trace(go.Pie(labels=labels, values=list(total_dict["logistic"]["values"]), name="Logistic"),
                1, 1)
    fig.add_trace(go.Pie(labels=labels, values=list(total_dict["svm"]["values"]), name="SVM"),
                1, 2)
    fig.update_traces(hole=.4, hoverinfo="label+percent+name", textinfo='label+percent')
    fig.append_trace(trace1, 2, 1)
    fig.append_trace(trace2,2,1)

    fig.update_layout(title_text="Model Performance", barmode="group")
    
    fig.update_layout(font=dict(size=13))
    #fig.update_annotations(font_size=20)
    fig.show()

generate_viz()



logistic: {'accuracy': 0.8534686005913825, 'sensitivity': 0.06987608695652174, 'specificity': 0.9938786035682468, 'precision': 0.6716397638823591}
svm: {'accuracy': 0.8482774021577443, 'sensitivity': 0.006210869565217392, 'specificity': 0.9991652198572499, 'precision': 0.5714}


**Save Model**

In [14]:
import pickle
#Final Model
model = logisitic_model.fit(main_df, ten_year_chd)
# Save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

## Predict Model

**Variable Declarations**

In [11]:
captions = ['Gender\n Male - 0 & Female - 1: ',
            'Age: ',
            'Education\n Less than High School - 1\n High School - 2\n College - 3\n University - 4: ',
            'Are you a Smoker\n No - 0 & Yes - 1: ',
            'Cigarettes per day: ',
            'Are you on BP Medications\n No - 0 & Yes - 1: ',
            'Do you have Strokes\n No - 0 & Yes - 1: ',
            'Do you have Hypertention\n No - 0 & Yes - 1: ',
            'Do you have Diabetes\n No - 0 & Yes - 1: ',
            'Total Cholestrol: ',
            'Systolic BP: ',
            'Diastolic BP: ',
            'BMI: ',
            'Heart Rate: ',
            'Glucose: ']
inputArr = []

**Input Validations**

In [10]:
# Checks if input is numeric
def isNumericInput(val):
  try:
    int(val)
    return int(val)
  except ValueError:
    print('\nINVALID INPUT\n')
    return -1

# Checks if input is a valid entry
def isValidRange(idx, val):
  minVal = main_df.iloc[:, idx].min()
  maxVal = main_df.iloc[:, idx].max()
  if val >= minVal and val <= maxVal:
    return val
  else:
    print('\nINVALID INPUT\n')
    return -1

**Input Function**

In [12]:
def getUserInputs():
  for idx, val in enumerate(captions):
    var = input(captions[idx])
    print('\n')
    inputVal = isNumericInput(var)
    if inputVal > -1:
      range = isValidRange(idx, inputVal)
      if range > -1:
        inputArr.append(inputVal)
      else:
        break
    else:
      break
  if len(inputArr) != len(main_df.columns):
    del inputArr[:]
    getUserInputs()

**Prediction Function**

In [13]:
def modelPrediction():

  X_Predict = pd.DataFrame([inputArr], columns = main_df.columns)
  print('Input Parameters:\n',X_Predict)

  # Load the model from disk
  loaded_model = pickle.load(open(filename, 'rb'))

  predict_output = loaded_model.predict(X_Predict)
  if predict_output == 1:
    print('\nPatient has 10-year risk of future Coronary Heart Disease')
  else:
    print('\nPatient does not have 10-year risk of future Coronary Heart Disease')

**Accept User Input and  Predict**

In [15]:
print('::::::::Prediction for Coronary Heart Disease::::::::')
print('\n\nEnter Health Parameters:\n')
getUserInputs()
modelPrediction()

::::::::Prediction for Coronary Heart Disease::::::::


Enter Health Parameters:

Gender
 Male - 0 & Female - 1: 0


Age: 50


Education
 Less than High School - 1
 High School - 2
 College - 3
 University - 4: 3


Are you a Smoker
 No - 0 & Yes - 1: 1


Cigarettes per day: 40


Are you on BP Medications
 No - 0 & Yes - 1: 1


Do you have Strokes
 No - 0 & Yes - 1: 0


Do you have Hypertendion
 No - 0 & Yes - 1: 1




KeyboardInterrupt: ignored